# 1. 기본 세팅

In [1]:
import pandas as pd
import numpy as np

import os
import glob
from datetime import date
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F

- 모델의 epoch가 끝나면 log를 logs에 저장하여 훈련 과정을 확인하고, epoch당 model을 weights에 저장함.

In [2]:
## logs와 weights 폴더가 없으면 생성함

if glob.glob('logs') != ['logs']:
    os.mkdir('logs')
else:
    pass

if glob.glob('weights') != ['weights']:
    os.mkdir('weights')
else:
    pass

In [3]:
# log_model에 훈련하는 모델의 대표명을 설정하여 저장

log_model = 'nmf'

today = date.today()
log_formatter = logging.Formatter("%(asctime)s %(message)s")
logger = logging.getLogger()

log_file_name = "./logs/{}_{}".format(today, log_model)

file_handler = logging.FileHandler("{}.log".format(log_file_name))
file_handler.setFormatter(log_formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.DEBUG)

# 2. 데이터 전처리

- 무비렌즈의 데이터로 실습
- 파일 다운로드 링크 : http://files.grouplens.org/datasets/movielens/ml-1m.zip
- 파일 설명 다운로드 링크 : http://files.grouplens.org/datasets/movielens/ml-1m-README.txt

In [4]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2020-07-08 14:16:07--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  2.60MB/s    in 2.2s    

2020-07-08 14:16:10 (2.60 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [5]:
## ml-1m이라는 폴더의 ratings를 사용함
!unzip ml-1m.zip

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [6]:
pd.read_csv('./ml-1m/ratings.dat').head()

,1::1193::5::978300760
0,1::661::3::978302109
1,1::914::3::978301968
2,1::3408::4::978300275
3,1::2355::5::978824291
4,1::1197::3::978302268


In [7]:
## 파일 설명에 따라 ratings의 columns는 'UserID, MovieID, Rating, Timestamp' 으로 설정한다.
ratings_cols = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv('./ml-1m/ratings.dat', sep='::', engine='python', names=ratings_cols)
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


# 3. 모델 구성

- pytorch 사용을 위한 dataloader, model 세팅

In [8]:
num_users = ratings.UserID.unique().shape[0]
num_items = ratings.MovieID.unique().shape[0]
print('유저의 수 : %d, 영화의 수 : %d' %(num_users, num_items))

유저의 수 : 6040, 영화의 수 : 3706


In [9]:
## UserID는 users, MovieID는 items, Rating은 y로 구성된 데이터셋 클래스 생성

from torch.utils.data import Dataset, TensorDataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, users, items, y):
        self.x = torch.cat([
            torch.LongTensor(users).unsqueeze(0).transpose(0, 1),
            torch.LongTensor(items).unsqueeze(0).transpose(0, 1)
        ], axis=1)
        self.y = torch.FloatTensor(y)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [10]:
## LabelEncoder를 사용하여 유저와 아이템을 0부터 labeling하여 데이터 train_dataset을 구성

from sklearn import preprocessing
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()

batch_size = 256

train_dataset = CustomDataset(
    le1.fit_transform(ratings.UserID),
    le2.fit_transform(ratings.MovieID),
    ratings.Rating.values
)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [11]:
## 위에서 확인한 유저의 수, 영화의 수 그리고 embedding할 차원을 설정하여 모델을 구성
## 모델은 유저와 영화의 임베딩 값을 dot product하여 rating을 계산하는 구조로 되어 있음

class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=128):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        x, y = data[:,:1], data[:,1:]
        u, v = self.user_emb(x), self.item_emb(y)
        return (u.squeeze(1) * v.squeeze(1)).sum(1)

In [12]:
## gpu가 있는 경우 gpu 연산을 사용하여 모델을 생성

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print('사용할 device : %s' % device)

사용할 device : cuda


In [13]:
model = MF(num_users, num_items, emb_size=128).to(device)
print('모델 구성 :\n\n', model)

모델 구성 :

 MF(
  (user_emb): Embedding(6040, 128)
  (item_emb): Embedding(3706, 128)
)


In [14]:
## optimizer, learning rate,  loss function을 설정
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss()

In [15]:
## 훈련 모드로 설정
model.train()

## epoch를 10으로 설정하고 모델 훈련
EPOCHS = 10

for e in range(EPOCHS):
    print('start : '+str(e)+' epoch')
    avg_loss = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = loss_fn(y_hat, y)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
    torch.save(model.state_dict(), "./weights/{}_{}.pt".format(log_model,e+1))
    print("e{}: loss: {}".format(e+1, loss/(batch_idx+1)))
    logger.info("e{}: loss: {}".format(e+1, loss/(batch_idx+1)))
print('training complete')

start : 0 epoch
e1: loss: 0.0007324457983486354
start : 1 epoch
e2: loss: 0.0006471379310823977
start : 2 epoch
e3: loss: 0.0003961047332268208
start : 3 epoch
e4: loss: 0.0004213388019707054
start : 4 epoch
e5: loss: 0.0007404053467325866
start : 5 epoch
e6: loss: 0.0007485683308914304
start : 6 epoch
e7: loss: 0.0011520812986418605
start : 7 epoch
e8: loss: 0.0011720028705894947
start : 8 epoch
e9: loss: 0.0011067170416936278
start : 9 epoch
e10: loss: 0.0009063336765393615
training complete


# 4. 모델 사용

In [16]:
## 평가모드로 모델을 변환하여 사용

model.eval()

MF(
  (user_emb): Embedding(6040, 128)
  (item_emb): Embedding(3706, 128)
)

In [17]:
## train data를 활용한 훈련 결과 검증을 위한 data 생성
ratings['le_UserID'] = le1.fit_transform(ratings.UserID)
ratings['le_MovieID'] = le1.fit_transform(ratings.MovieID)

ratings

,UserID,MovieID,Rating,Timestamp,le_UserID,le_MovieID
0,1,1193,5,978300760,0,1104
1,1,661,3,978302109,0,639
2,1,914,3,978301968,0,853
3,1,3408,4,978300275,0,3177
4,1,2355,5,978824291,0,2162
...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,6039,1019
1000205,6040,1094,5,956704887,6039,1022
1000206,6040,562,5,956704746,6039,548
1000207,6040,1096,4,956715648,6039,1024


In [18]:
## LongTensor로 변환하고 model에 넣어 계산된 rating 예측

val_tensor = torch.LongTensor(ratings[['le_UserID', 'le_MovieID']].values).to(device)
pred = model(torch.LongTensor(ratings[['le_UserID', 'le_MovieID']].values).to(device))

In [19]:
## 예측 값을 ratings DataFrame에 추가
ratings['pred_Rating'] = pred.to('cpu').detach().numpy()

In [20]:
ratings

,UserID,MovieID,Rating,Timestamp,le_UserID,le_MovieID,pred_Rating
0,1,1193,5,978300760,0,1104,4.341512
1,1,661,3,978302109,0,639,3.178971
2,1,914,3,978301968,0,853,1.895466
3,1,3408,4,978300275,0,3177,6.053298
4,1,2355,5,978824291,0,2162,4.379835
...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,6039,1019,1.172910
1000205,6040,1094,5,956704887,6039,1022,4.649513
1000206,6040,562,5,956704746,6039,548,5.559851
1000207,6040,1096,4,956715648,6039,1024,4.292498


## 5. 저장된 모델을 불러와서 사용하기

- 커널을 종료하고 새롭게 모델을 불러와서 사용한다는 가정

In [21]:
import pandas as pd
import numpy as np

import os
import glob
from datetime import date
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
## 모델 클래스를 정의함

class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=128):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        x, y = data[:,:1], data[:,1:]
        u, v = self.user_emb(x), self.item_emb(y)
        return (u.squeeze(1) * v.squeeze(1)).sum(1)

In [23]:
## gpu가 있는 경우 gpu 연산을 사용하여 모델을 평가

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print('사용할 device : %s' % device)

사용할 device : cuda


In [24]:
## 저장한 모델 파일의 경로를 지정하고 모델을 불러옴
## 모델 훈련에서 사용한 '유저의 수 : 6040, 영화의 수 : 3706'의 값과 임베딩 차원을 입력

model = MF(6040, 3706, emb_size=128).to(device)

model_path = './weights/nmf_10.pt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

* 모델을 불러온 후에는 위 4번 항목과 사용하는 방법이 동일함